In [2]:
!pip install keras==2.2.4

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
     |████████████████████████████████| 312 kB 821 kB/s eta 0:00:01


In [4]:
!pip install gensim

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
     |████████████████████████████████| 24.2 MB 16.8 MB/s eta 0:00:01��█████████████████       | 18.9 MB 16.8 MB/s eta 0:00:01�███████████████████████▍    | 20.8 MB 16.8 MB/s eta 0:00:01
     |████████████████████████████████| 103 kB 97.2 MB/s eta 0:00:01
     |████████████████████████████████| 1.4 MB 3.3 MB/s eta 0:00:01
     |████████████████████████████████| 128 kB 3.2 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 39.9 MB/s eta 0:00:01
     |████████████████████████████████| 118 kB 3.2 MB/s eta 0:00:01
     |████████████████████████████████| 6.2 MB 1.6 MB/s eta 0:00:01
     |████████████████████████████████| 69 kB 66.6 MB/s eta 0:00:01
     |████████████████████████████████| 547 kB 45.4 MB/s eta 0:00:01
  Created wheel for smart-open: filename=smart_open-2.0.0-py3-none-any.whl size=101341 sha256=b6cda6ae750a3f40346ee1b7d2740130343e1af4dee36858b2e91975ca4f2f35
  Stored in directory: /home/tione/.cache/pip/

In [14]:
import tensorflow

In [15]:
tensorflow.__version__

'1.14.0'

In [5]:
from keras.models import Model
from keras.optimizers import SGD,Adam,RMSprop
from keras.layers import *
import os
from keras import backend as K
from keras.engine.topology import Layer
import h5py
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import pandas as pd
import numpy as np
import keras

from gensim.models import Word2Vec

In [2]:
keras.__version__

'2.2.4'

In [6]:
#coding=utf-8
'''
Single model may achieve LB scores at around 0.043
Don't need to be an expert of feature engineering
All you need is a GPU!!!!!!!

The code is tested on Keras 2.0.0 using Theano backend, and Python 3.5

referrence Code:https://www.kaggle.com/lystdo/lstm-with-word2vec-embeddings
'''

########################################
## import packages
########################################
import os
import re
import csv
import codecs
import numpy as np
import pandas as pd

########################################
## set directories and parameters
########################################



from keras import backend as K
from keras.engine.topology import Layer
# from keras import initializations
from keras import initializers, regularizers, constraints

np.random.seed(2018)

class Attention(Layer):
    def __init__(self,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        """
        Keras Layer that implements an Attention mechanism for temporal data.
        Supports Masking.
        Follows the work of Raffel et al. [https://arxiv.org/abs/1512.08756]
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:

        """
        self.supports_masking = True
        # self.init = initializations.get('glorot_uniform')
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias

        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        self.step_dim = input_shape[1]
        assert len(input_shape) == 3 # batch ,timestep , num_features
        print(input_shape)
        self.W = self.add_weight((input_shape[-1],), #num_features
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),#timesteps
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim
        print(K.reshape(x, (-1, features_dim)))# n, d
        print(K.reshape(self.W, (features_dim, 1)))# w= dx1
        print(K.dot(K.reshape(x, (-1, features_dim)), K.reshape(self.W, (features_dim, 1))))#nx1

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)), K.reshape(self.W, (features_dim, 1))), (-1, step_dim))#batch,step
        print(eij)
        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        print(a)
        a = K.expand_dims(a)
        print("expand_dims:")
        print(a)
        print("x:")
        print(x)
        weighted_input = x * a
        print(weighted_input.shape)
        return weighted_input
#         return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
#         return input_shape[0], input_shape[-1]
        return input_shape[0],self.step_dim, self.features_dim


In [11]:
def buid_model():
    
#     w2v_model = Word2Vec.load("../get_w2v_feat/word2vec.model")
    
    S_inputs = Input(shape=(200,),name='main_input',dtype='int32')

    # embeddings = Embedding(input_dim=380,output_dim=16,input_length=200,mask_zero = False)(S_inputs)
    embeddings = Embedding(input_dim=3812203,output_dim=16,name = 'emb',mask_zero = True)(S_inputs)
#     O_seq = Bidirectional(LSTM(16,activation='softsign',return_sequences=True))(embeddings)
    O_seq = Attention()(embeddings)
#     O_seq = Bidirectional(LSTM(16,activation='softsign',return_sequences=False))(O_seq)
    O_seq = GlobalAveragePooling1D()(O_seq)
#     O_seq = GlobalMaxPool1D()(O_seq)
    outputs = Dense(10, activation='softmax', name='main_output')(O_seq)

    # 定义一个具有两个输入输出的模型
    model = keras.models.Model(inputs=[S_inputs],#,auxiliary_input],
                               outputs=[outputs])  # 这里的输入输出顺序与fit时一致就好
    
#     model.layers[1].trainable = False
    
#     opt = RMSprop(lr=0.01,  clipnorm=1.0)
    opt = Adam(lr=0.01)
    model.compile(optimizer=opt,
                  sample_weight_mode='None',#"temporal",
                  loss={'main_output': 'categorical_crossentropy'},
                 metrics=['accuracy'])
    print(model.summary())
    return model

#     print(model.summary())


In [23]:
def data_load():
    print('loading data ... \n')

    with h5py.File('../word_train_ad.h5', 'r') as f:
        data = np.array(f.get('word_data'))

    label = pd.read_csv('../../../train_preliminary/user.csv').sort_values(by=['user_id'])

    train_x, test_x, train_y, test_y = train_test_split(data, label, test_size=0.2, random_state=2020)

    train_y_age = train_y['age'].values - 1
    train_y_age = keras.utils.np_utils.to_categorical(train_y_age, num_classes=10)
    train_y_gender = train_y['gender'].values - 1

    test_y_age = test_y['age'].values - 1
    test_y_age = keras.utils.np_utils.to_categorical(test_y_age, num_classes=10)
    test_y_gender = test_y['gender'].values - 1

    print('get data ... \n')

    return train_x, test_x, train_y_age, train_y_gender,test_y_age,test_y_gender

In [12]:
def get_filename_for_saving(save_dir):
    return os.path.join(save_dir,
                        "self_lstm_attention_max_dense_ad_age_adm_{val_loss:.3f}-{val_acc:.3f}-{epoch:03d}-{loss:.3f}-{acc:.3f}.hdf5")


model = buid_model()
print('lstm model geted...\n')

(None, 200, 16)
Tensor("attention_2/Reshape:0", shape=(?, 16), dtype=float32)
Tensor("attention_2/Reshape_1:0", shape=(16, 1), dtype=float32)
Tensor("attention_2/MatMul:0", shape=(?, 1), dtype=float32)
Tensor("attention_2/Reshape_6:0", shape=(?, 200), dtype=float32)
Tensor("attention_2/truediv:0", shape=(?, 200), dtype=float32)
expand_dims:
Tensor("attention_2/ExpandDims:0", shape=(?, 200, 1), dtype=float32)
x:
Tensor("emb_1/embedding_lookup/Identity:0", shape=(?, 200, 16), dtype=float32)
(?, 200, 16)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_input (InputLayer)      (None, 200)               0         
_________________________________________________________________
emb (Embedding)              (None, 200, 16)           60995248  
_________________________________________________________________
attention_2 (Attention)      (None, 200, 16)           216       
_____________________________

In [20]:
train_x, test_x, train_y_age, train_y_gender,test_y_age,test_y_gender = data_load()

loading data ... 

get data ... 



In [25]:
print('lstm model fit...\n')
checkpointer = keras.callbacks.ModelCheckpoint(
    filepath=get_filename_for_saving(''),
    save_best_only=False)
stopping = keras.callbacks.EarlyStopping(patience=8)
reduce_lr = keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=2, min_lr=0.0001)

# with h5py.File('../train_data_weight.h5', 'r') as f:
#         weight = np.array(f.get('weight'))
# train_w, test_w= train_test_split(weight, test_size=0.2, random_state=2020)

# train_w = np.squeeze(train_w)
# t_w = 10/np.log(train_w)


model.fit({'main_input': train_x },#,'aux_input': train_x_sta},
          {'main_output': train_y_age},
          epochs=100,
          batch_size=256,
#           class_weight = {0:0.698, 1:0.520, 2:0.671, 3:0.698, 4:0.945,5:0.890, 6:0.986, 7:1. , 8:0.808,9:0.657},
#           class_weight = {0:1.,1:2.159}
#           sample_weight = t_w,
          validation_data=({'main_input': test_x},#,'aux_input': test_x_sta},
                           {'main_output': test_y_age}),
          callbacks=[checkpointer, reduce_lr, stopping])

lstm model fit...

Train on 720000 samples, validate on 180000 samples
Epoch 1/100
 72960/720000 [==>...........................] - ETA: 38:28 - loss: 2.0643 - acc: 0.2255

KeyboardInterrupt: 